# Lecture 6 -  A brief introduction to R and RCall 

Lecture 6 will be delivered Philippe Blanchard from [PAIS](https://warwick.ac.uk/fac/soc/pais/) (Politics and International Studies) and [Q-Step](https://warwick.ac.uk/fac/cross_fac/q-step). In this short tutorial I will give a quick introduction to some computational techniques that he will be using in his lecture. 

When we chose Julia as the programming language for the IL027 module it was because it provides a natural, easy to learn, syntax. In particular it makes it possible to perform non-trivial tasks with fairly elementary programming paradigms. E.g., no aspects of object oriented of functional programming are required. However, in many disciplines there are specific language that dominate, primarily due to access to specific libraries. For example, in many branches of computational modelling, scientific computing and numerical analysis, Matlab and Python have become the dominant players. In Statistics and Data Analysis on the other hand, the language [R](https://www.r-project.org) is the clear industry standard.

Philippe's lecture will require the use of some of these highly specialised R libraries which provide functionality that is not easy to replicate in Julia. Instead, we will simply call these libraries from our Julia notebooks. The purpose of this introductory tutorial is to show you how this can be done and talk a little bit about what happens behind the scene. 

Interested Students should also look at the related [`PyCall`](https://github.com/JuliaPy/PyCall.jl) package. There are also similar packages for Matlab, Ruby, Java, C++ and many other languages, but we don't provide these interfaces on our Jupyter server. C integration is built in; cf. `ccall`.

In [ ]:
# First, we load the `RCall` library which provides the interface to `R`
using RCall

After executing `using RCall` an entire runtime system of `R` is running in the background, which we don't see but to which we can send commands. For example, to create a vector in `R` and then sum over its components is done as follows:
```{R}
x <- c(1,2,3,4,5,6,7,8,9,10)
s <- sum(x)
```
In Julia we can execute this short code snippet as follows:

In [ ]:
R"""
x <- c(1,2,3,4,5,6,7,8,9,10)
s <- sum(x)
"""

In [ ]:
# now the variables `x, s` "live"  in R and not in Julia: 
x

In [ ]:
#  A more convenient way therefore to perform actions in R and keep 
# the results is as follows:
x = R"""c(1,2,3,4,5,6,7,8,9,10)"""

We see that `x` is stored as an `RObject`, i.e. it is just a "link"
to a vector stored in the `R` runtime system.  What is very nice about 
the `RCall` library is that it lets us use `x` much in the same way 
as if it were a Julia object. The integration is really quite seemless:

In [ ]:
sum(x)

The overarching strategy how to use `RCall` is therefore to 
perform only those operations in `R` that are not available in 
pure `Julia`, but try to keep the majority of operations in 
pure Julia. The rest of this notebooks shows a minimal collection of 
functionality provided by `RCall`, including: 

* `library` to load `R` libraries
* `rcopy` to automatically convert R objects into Julia objects 
* `robject` : convert a Julia object to an R object 
* `rcall` : call `R` functions 
* Interpolation of julia values into an R command / function call 

For more information there is a nice introduction in the [`RCall` Documentation](http://juliainterop.github.io/RCall.jl/stable/gettingstarted)

In [ ]:
# following on from above - if we prefer to keep `x` as 
# a Julia array rather than an `R` array, then we can do 
xj = rcopy(x)   #  xj is now a copy of `x` living in the Julia runtime

In [ ]:
# load the R library `boot`
R"""library(boot)"""
# and load a dataset 
cityr = R"""boot::city"""

In [ ]:
# or import the same dataset into Julia as a DataFrame
using DataFrames   
cityj = rcopy(R"""boot::city""")

In [ ]:
cityj[!, :u]

In [ ]:
# calling an `R` function called `summary`
rcall(:summary, cityr)

In [ ]:
# same as the last cell, but maybe more readable? 
# use the $ interpolation to evaluate `cityr` inside `R`
R"""summary($cityr)"""

In [ ]:
# convert a Julia  object back to R 
city2 = robject(cityj)

In [ ]:
# some standard R datasets have been made available using the following package:
using RDatasets 
# load one of them 
mtcars = dataset("datasets", "mtcars")

In [ ]:
# load R's famous ggplot library 
R"""library(ggplot2)"""
R"""ggplot($mtcars, aes(x = WT, y = MPG)) + geom_point()"""